# Analyse exploratoire des données

In [1]:
import os
import glob
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession

# Initialiser une session Spark
spark = SparkSession.builder \
    .appName("SQL with PySpark") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .master("local[*]") \
    .getOrCreate()


# Obtenir la date actuelle au format YYYYMMDD
current_date = datetime.now().strftime("%Y%m%d")

# Spécifier le chemin du dossier "backup"
backup_dir = 'backup'

# Vérifier si le dossier "backup" existe
if not os.path.exists(backup_dir):
    print(f"Le dossier {backup_dir} n'existe pas. Veuillez le créer et y ajouter des fichiers.")
    exit()

# Rechercher tous les fichiers correspondant au format "properties_db_*.csv" dans le dossier backup
file_pattern = os.path.join(backup_dir, "properties_db_*.csv")
files = glob.glob(file_pattern)

if files:
    # Filtrer pour trouver un fichier de la date courante ou le plus récent
    current_date_pattern = os.path.join(backup_dir, f"properties_db_{current_date}.csv")
    if current_date_pattern in files:
        latest_file = current_date_pattern
        print(f"Fichier correspondant à la date courante trouvé : {latest_file}")
    else:
        # Si aucun fichier pour la date courante, sélectionner le fichier le plus récent
        latest_file = max(files, key=os.path.getmtime)
        print(f"Aucun fichier pour la date courante. Dernier fichier disponible : {latest_file}")

    # Charger le fichier CSV dans un DataFrame
    try:
        pandas_data_properties = spark.read.csv(latest_file, header=True, inferSchema=True) 
        print("Fichier importé avec succès.")
        # Afficher les 5 premières lignes
        pandas_data_properties.show(5)

        # Afficher le schéma des données
        pandas_data_properties.printSchema()# Informations sur le DataFrame
        # Enregistrer le DataFrame comme une table temporaire
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {latest_file}: {e}")
else:
    print("Aucun fichier disponible dans le dossier 'backup'.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/07 21:35:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Fichier correspondant à la date courante trouvé : backup/properties_db_20250207.csv


Fichier importé avec succès.
+--------+----------+-------+--------------------+------------+-----------+--------+------+------------------+-----------------+-----------------+-------+----------+------+----------+-----------+------+--------------------+--------------------+--------------------+------+--------------------+-------------------+--------------------+-----------+
|     _id|  add_date|   city|         description|   longitude|   latitude|   price|  type|   municipal_taxes|     school_taxes|construction_year|surface|    region|vendue|nb_bedroom|nb_bathroom|revenu|             img_src|             address|           update_at|unites|                link|           category|              street|last_update|
+--------+----------+-------+--------------------+------------+-----------+--------+------+------------------+-----------------+-----------------+-------+----------+------+----------+-----------+------+--------------------+--------------------+--------------------+------+-----

In [2]:
from pyspark.sql.functions import year, month, to_date
# Ajouter les colonnes pour l'année et le mois
df_properties = (
    pandas_data_properties
    .withColumn("add_year", year("add_date"))   # Année d'ajout
    .withColumn("add_month", month("add_date"))  # Mois d'ajout
)

In [3]:
# Enregistrer le DataFrame comme une table temporaire
df_properties.createOrReplaceTempView("properties")

25/02/07 21:36:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
query="select * from properties where _id=23985786;"
spark.sql(query).show()

+--------+----------+--------------+--------------------+----------+---------+--------+-------+------------------+-----------------+-----------------+-------+--------+------+----------+-----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------+--------+---------+
|     _id|  add_date|          city|         description| longitude| latitude|   price|   type|   municipal_taxes|     school_taxes|construction_year|surface|  region|vendue|nb_bedroom|nb_bathroom| revenu|             img_src|             address|           update_at|              unites|                link|     category|              street|last_update|add_year|add_month|
+--------+----------+--------------+--------------------+----------+---------+--------+-------+------------------+-----------------+-----------------+-------+--------+------+----------+-----------+-------+--------------------+--------------------

## Muiltilogement

In [5]:
category = 'Multilogement'
type = 'Triplex'
city='Shawinigan'
vendue = None #'False'

where= f"""
WHERE 
"""
if category:
    where += f"category = '{category}'"
if city:
    where += f" AND city = '{city}'"
if vendue:
    where += f" AND vendue = {vendue}"
if type:
    where += f" AND type = '{type}'"


multilogements_properties_query = f"""
SELECT _id,
     price,
     surface,
     municipal_taxes,
     school_taxes,
     longitude,
     latitude,
     link,
     street,
     img_src,
     city,
     add_date,
     year(add_date) AS add_year,
     month(add_date) AS add_month,
     last_update,
     year(last_update) AS sold_from_year,
     month(last_update) AS sold_from_month,
     cast(construction_year AS INT) AS construction_year,
     revenu,
     category,
     type AS property_type,
     vendue,
     unites
FROM properties
{where}
ORDER BY add_year, add_month;
"""
multilogements_properties = spark.sql(multilogements_properties_query)

# Afficher les résultats
multilogements_properties.show()



+--------+--------+-------+------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+----------+----------+--------+---------+-----------+--------------+---------------+-----------------+-------+-------------+-------------+------+--------------------+
|     _id|   price|surface|   municipal_taxes|      school_taxes|   longitude|   latitude|                link|              street|             img_src|      city|  add_date|add_year|add_month|last_update|sold_from_year|sold_from_month|construction_year| revenu|     category|property_type|vendue|              unites|
+--------+--------+-------+------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+----------+----------+--------+---------+-----------+--------------+---------------+-----------------+-------+-------------+-------------+------+--------------------+
|24956374|195000.0|37999.0|            8

In [6]:

multilogements_properties.createOrReplaceTempView("multilogements_properties")

In [7]:
query = f"""
WITH properties_data AS (
    SELECT add_year,
           add_month,
           category,
           property_type,
           COUNT(*) AS nbr_properties_added,
           SUM(CASE WHEN vendue = True THEN 1 ELSE 0 END) AS nbr_properties_vendu,
           SUM(CASE WHEN vendue = False THEN 1 ELSE 0 END) AS nbr_properties_en_vente
    FROM multilogements_properties 
    GROUP BY add_year, add_month, category, property_type
)
SELECT add_year,
       add_month,
       category,
       property_type,
       nbr_properties_added,
       nbr_properties_vendu,
       nbr_properties_en_vente
FROM properties_data
ORDER BY add_year, add_month;
"""
result = spark.sql(query)

# Afficher les résultats
result.show()

+--------+---------+-------------+-------------+--------------------+--------------------+-----------------------+
|add_year|add_month|     category|property_type|nbr_properties_added|nbr_properties_vendu|nbr_properties_en_vente|
+--------+---------+-------------+-------------+--------------------+--------------------+-----------------------+
|    2023|        2|Multilogement|      Triplex|                  13|                  13|                      0|
|    2023|        4|Multilogement|      Triplex|                   2|                   2|                      0|
|    2023|        5|Multilogement|      Triplex|                   4|                   4|                      0|
|    2023|        6|Multilogement|      Triplex|                   6|                   6|                      0|
|    2023|        7|Multilogement|      Triplex|                   2|                   2|                      0|
|    2024|        4|Multilogement|      Triplex|                  12|           

In [8]:
pandas_data = multilogements_properties.toPandas()

### 🔥 Carte Heatmap des Prix Immobiliers 🏡📊

✅ Affichage du Prix et Adresse au survol (tooltip)

✅ Popup avec image, prix et adresse 🏡

✅ Heatmap pour visualiser la concentration des ventes 🔥

✅ Marqueurs rouges pour vendus, verts pour disponibles 🟩🟥

✅ Regroupement automatique des points proches (clusters)



In [9]:
import folium
from folium.plugins import MarkerCluster, HeatMap

# Charger tes données (remplace par ton DataFrame PySpark converti en Pandas)
pandas_data = pandas_data.dropna(subset=["latitude", "longitude", "price", "city", "vendue"])  # Supprimer les NaN

# Définir le centre de la carte
center_lat = pandas_data["latitude"].mean()
center_lon = pandas_data["longitude"].mean()

# Créer la carte
map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Ajouter un cluster de marqueurs
marker_cluster = MarkerCluster().add_to(map)

# Liste pour stocker les coordonnées de la heatmap (uniquement les maisons vendues)
heat_data = []

# Boucle pour ajouter les marqueurs et préparer la heatmap
for _, row in pandas_data.iterrows():
    color = "red" if row["vendue"] == True else "green"  # Déterminer la couleur 
    image_url = row["img_src"] if "img_src" in pandas_data.columns and row["img_src"] \
    else "https://via.placeholder.com/150"
    popup_content = f"""
        <div style="width: 200px;">
            <a href='{row['link']}'><b> {row['_id']} </a></b><br>
            <b> {row['price']:,.0f}$ </b><br>
            <b>{row.get('street')}, {row.get('city')}</b><br>
            <b> {row.get('surface')} m²</b><br>
            revenu : <b>{row.get('revenu')}$</b><br>
            Année: <b> {row.get('construction_year')}</b><br>
            ajouté le : <b> {row.get('add_date')}</b><br>
            vendue le : <b> {row.get('last_update')}</b><br>
            <img src="{image_url}" alt="Image" width="200px">
        </div>
        """
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=folium.Popup(popup_content, max_width=250),
        tooltip=f"${row['price']:,.0f}",
        icon=folium.Icon(color=color),
    ).add_to(marker_cluster)

    # Ajouter les points vendus à la heatmap
    if row["vendue"] == True:
        heat_data.append([row["latitude"], row["longitude"], row["price"]])  # Pondération avec le prix

# Ajouter la Heatmap (densité des maisons vendues)
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(map)

# Afficher la carte
map


✅ Animation de l'évolution des ventes avec une timeline ⏳

✅ Popup avec Prix, Adresse et Image 🏡

✅ Marqueurs colorés (rouge = vendu, vert = dispo) 🟩🟥

✅ Glisser sur la timeline pour voir les ventes par mois 📆

✅ Auto-play et possibilité de boucler l'animation 🎥

In [10]:

from folium.plugins import TimestampedGeoJson

# Convertir add_date en datetime
pandas_data["add_date"] = pd.to_datetime(pandas_data["add_date"])

# Créer la carte centrée sur la moyenne des latitudes/longitudes
center_lat = pandas_data["latitude"].mean()
center_lon = pandas_data["longitude"].mean()
map_animated = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Fonction pour formater les données GeoJSON animées
def create_geojson_features(df):
    features = []
    
    for _, row in df.iterrows():
        color = "red" if row["vendue"] == True else "green"
        image_url = row["img_src"] if "img_src" in df.columns else "https://via.placeholder.com/150"
        popup_content = f"""
        <div style="width: 200px;">
            <a href='{row['link']}'><b> {row['_id']} </a></b><br>
            <b> {row['price']:,.0f}$ </b><br>
            <b>{row.get('street')}, {row.get('city')}</b><br>
            <b> {row.get('surface')} m²</b><br>
            <b>revenu : {row.get('revenu')}$</b><br>
            Année: <b> {row.get('construction_year')}</b><br>
            ajouté le : <b> {row.get('add_date')}</b><br>
            {'vendue le : <b> ' + str(row.get('last_update')) + '</b><br>' if row["vendue"] == True else ''}
            <img src="{image_url}" alt="Image" width="200px">
        </div>
        """

        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row["longitude"], row["latitude"]],
            },
            "properties": {
                "time": row["add_date"].isoformat(),
                "popup": popup_content,
                "style": {"color": color},
                "icon": "circle",
                "iconstyle": {
                    "fillColor": color,
                    "fillOpacity": 0.6,
                    "stroke": False,
                    "radius": 5,
                },
            },
        }
        features.append(feature)
    
    return features

# Générer les données GeoJSON
geojson_data = {
    "type": "FeatureCollection",
    "features": create_geojson_features(pandas_data),
}

# Ajouter l'animation des ventes par mois
TimestampedGeoJson(
    geojson_data,
    period="P1M",  # Animation par mois
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options="YYYY-MM-DD",
    time_slider_drag_update=True,
).add_to(map_animated)

# Afficher la carte
map_animated

In [11]:
# Charger les données
#pandas_data = pandas_data.dropna(subset=["latitude", "longitude", "price"])

# Créer la carte centrée sur la moyenne des latitudes/longitudes
center_lat = pandas_data["latitude"].mean()
center_lon = pandas_data["longitude"].mean()
map_heatmap = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Ajouter la heatmap (poids = prix)
heat_data = [
    [row["latitude"], row["longitude"], row["price"]] for _, row in pandas_data.iterrows()
]
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(map_heatmap)

# Affichage de la carte
map_heatmap

### 📊 Mieux comprendre la variable price (Prix) 💰

#### 1️⃣ Analyse Statistique Globale

In [12]:


# Afficher les statistiques de base
print(pandas_data["price"].describe())

count        87.000000
mean     321086.172414
std      165807.694718
min      129000.000000
25%      212450.000000
50%      279500.000000
75%      349450.000000
max      999500.000000
Name: price, dtype: float64


#### 2️⃣ Détection des Valeurs Extrêmes (Outliers)

In [13]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Calcul des quartiles
Q1 = pandas_data["price"].quantile(0.25)
Q3 = pandas_data["price"].quantile(0.75)
IQR = Q3 - Q1

# Définition des bornes
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrer les prix dans ces bornes
df_filtered = pandas_data[(pandas_data["price"] >= lower_bound) & (pandas_data["price"] <= upper_bound)]

# Vérifier la réduction des valeurs extrêmes
print(f"Avant : {len(pandas_data)} lignes, Après : {len(df_filtered)} lignes (Outliers supprimés)")

Avant : 87 lignes, Après : 79 lignes (Outliers supprimés)


In [14]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create subplots vertically
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Prix (avec outliers)",
                                  "Prix (après suppression des outliers)"))

# Add box plots
fig.add_trace(
    go.Box(x=pandas_data["price"], name="Avec outliers"),
    row=1, col=1
)

fig.add_trace(
    go.Box(x=df_filtered["price"], name="Sans outliers"),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title_text="Comparaison des distributions de prix",
    showlegend=False,
    height=600,  # Increased height for better vertical display
    xaxis_title="Prix ($)",
    xaxis2_title="Prix ($)"
)

fig.show()


3️⃣ Distribution des Prix

In [15]:
import plotly.express as px

fig = px.histogram(df_filtered, x="price", nbins=50, title="Distribution des prix des propriétés")
fig.update_layout(
    xaxis_title="Prix ($)",
    yaxis_title="Nombre de propriétés",
    bargap=0.1
)
fig.show()

In [16]:
import plotly.express as px
import plotly.graph_objects as go

# Convert add_date to datetime if not already done
pandas_data["add_date"] = pd.to_datetime(pandas_data["add_date"])

# Create quarterly average and median price trend
quarterly_data = pandas_data.groupby(pandas_data["add_date"].dt.to_period("Q").astype(str)).agg({
    "price": ["mean", "median", "count"]
}).reset_index()
quarterly_data.columns = ["quarter", "avg_price", "median_price", "count"]

# Create figure with both mean and median
fig = go.Figure()

# Add mean price line
fig.add_trace(go.Scatter(
    x=quarterly_data["quarter"],
    y=quarterly_data["avg_price"],
    name="Prix moyen",
    mode="lines+markers",
    hovertemplate="Prix moyen: %{y:,.0f}$<br>Nombre de propriétés: %{text}<extra></extra>",
    text=quarterly_data["count"]
))

# Add median price line
fig.add_trace(go.Scatter(
    x=quarterly_data["quarter"],
    y=quarterly_data["median_price"],
    name="Prix médian",
    mode="lines+markers",
    hovertemplate="Prix médian: %{y:,.0f}$<br>Nombre de propriétés: %{text}<extra></extra>",
    text=quarterly_data["count"]
))

# Update layout
fig.update_layout(
    title="Évolution des prix par trimestre",
    xaxis_title="Trimestre",
    yaxis_title="Prix ($)",
    hovermode="x unified"
)

fig.show()

In [17]:
pandas_data.columns

Index(['_id', 'price', 'surface', 'municipal_taxes', 'school_taxes',
       'longitude', 'latitude', 'link', 'street', 'img_src', 'city',
       'add_date', 'add_year', 'add_month', 'last_update', 'sold_from_year',
       'sold_from_month', 'construction_year', 'revenu', 'category',
       'property_type', 'vendue', 'unites'],
      dtype='object')

In [34]:
import ast# Fonction pour extraire les informations des unités
def extract_units_info(units_list):
    if pd.isna(units_list):
        return {}
    try:
        units_list = ast.literal_eval(units_list)
        if isinstance(units_list, list):
            units_info = {}
            for unit in units_list:
                unit_type = unit.get('unite')
            return units_info
    except (ValueError, SyntaxError):
        return {}
    return {}

# Appliquer la fonction pour extraire les informations des unités
pandas_data['units_info'] = pandas_data['unites'].apply(extract_units_info)

# Créer des colonnes distinctes pour chaque type d'unité
units_df = pandas_data['units_info'].apply(pd.Series).fillna(0)

print(units_df.columns)

# Regrouper "Autre" et "Other" en "Autre"
if 'Other' in units_df.columns:
    units_df['Autre'] += units_df['Other']
    units_df.drop(columns=['Other'], inplace=True)

# Ajouter les nouvelles colonnes au dataframe principal
pandas_data = pd.concat([pandas_data, units_df], axis=1)

# Supprimer la colonne temporaire 'units_info'
pandas_data.drop(columns=['units_info'], inplace=True)

# Afficher les premières lignes pour vérifier le résultat
print(pandas_data.head())

RangeIndex(start=0, stop=0, step=1)
        _id     price  surface  municipal_taxes  school_taxes  longitude  \
0  10415013  275000.0   6073.0           4513.0         146.0 -72.754186   
1  11479888  147000.0  19623.0           3107.0          78.0 -72.696142   
2  11545238  164900.0   5000.0           3826.0          88.0 -72.740379   
3  12857494  199000.0   6599.0           3571.0          79.0 -72.692128   
4  15451285  229000.0   7126.0           3614.0         118.0 -72.692276   

    latitude                                               link  \
0  46.522889  https://www.centris.ca/fr/triplex~a-vendre~sha...   
1  46.620876  https://www.centris.ca/fr/triplex~a-vendre~sha...   
2  46.552887  https://www.centris.ca/fr/triplex~a-vendre~sha...   
3  46.612182  https://www.centris.ca/fr/triplex~a-vendre~sha...   
4  46.610624  https://www.centris.ca/fr/triplex~a-vendre~sha...   

                        street  \
0          800 - 804, 111e Rue   
1          1428 - 1434, 4e Rue   
2 

25/02/08 08:29:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2862567 ms exceeds timeout 120000 ms
25/02/08 08:29:05 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/08 08:29:13 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [19]:
# Group columns by type for better organization
numeric_cols = pandas_data.select_dtypes(include=['int64', 'float64']).columns
datetime_cols = pandas_data.select_dtypes(include=['datetime64']).columns
object_cols = pandas_data.select_dtypes(include=['object']).columns
bool_cols = pandas_data.select_dtypes(include=['bool']).columns

# Create a summary dictionary
column_summary = {
    'Numeric Columns': list(numeric_cols),
    'Datetime Columns': list(datetime_cols),
    'Object/String Columns': list(object_cols),
    'Boolean Columns': list(bool_cols)
}

# Print the organized summary
for category, cols in column_summary.items():
    print(f"\n{category}:")
    for col in cols:
        print(f"- {col}")


Numeric Columns:
- price
- surface
- municipal_taxes
- school_taxes
- longitude
- latitude
- revenu

Datetime Columns:
- add_date

Object/String Columns:
- link
- street
- img_src
- city
- last_update
- category
- property_type
- unites

Boolean Columns:
- vendue


#### 6️⃣ Corrélations avec d'autres Variables

In [20]:
import plotly.express as px

correlation = pandas_data[["price", "surface", "longitude", "latitude", "municipal_taxes", "school_taxes", "revenu", "construction_year","add_date"]].corr()

fig = px.imshow(correlation, text_auto=True, aspect="auto", color_continuous_scale="RdBu")
fig.update_layout(title="Corrélations entre le prix et d'autres variables")
fig.show()

##### Scatter plot simple : Prix vs Revenu

In [21]:
import plotly.express as px

fig = px.scatter(pandas_data, 
                 x="revenu", 
                 y="price",
                 opacity=0.5,
                 title="Rélation entre le revenu et le prix",
                 labels={"revenu": "Revenu ($)", "price": "Prix ($)"},
                 hover_data=["city", "street"])

fig.update_yaxes(type="log")  # Échelle logarithmique pour les prix
fig.update_layout(showlegend=False)

fig.show()

#### Heatmap des prix par tranche de revenus

In [22]:
pandas_data["revenu_categorie"] = pd.qcut(pandas_data["revenu"], q=4, labels=["Bas", "Moyen-bas", "Moyen-haut", "Haut"])

fig = px.box(pandas_data, x="revenu_categorie", y="price", color="revenu_categorie", 
             title="Distribution des Prix par Catégorie de Revenu",
             labels={"revenu_categorie": "Catégorie de Revenu", "price": "Prix ($)"})

fig.update_layout(yaxis_title="Prix ($)", xaxis_title="Catégorie de Revenu")
# fig.update_yaxes(type="log")  # Uncomment if you want to use a logarithmic scale for the y-axis

fig.show()

In [23]:
import plotly.express as px

fig = px.scatter(pandas_data, 
                 x="surface", 
                 y="price",
                 opacity=0.5,
                 title="Rélation entre la surface et le prix",
                 labels={"surface": "Surface (m²)", "price": "Prix ($)"},
                 hover_data=["city", "street"])

fig.update_yaxes(type="log")  # Échelle logarithmique pour les prix
fig.update_layout(showlegend=False)

fig.show()

In [24]:
# mont_regie_maisons = df_properties[
#     (df_properties["category"] == "Maison individuelle") & (df_properties["region"] == "Montérégie")
# ]
# mont_regie_maisons.head(5)

In [25]:
# # Filtrer les données pour la catégorie "Maison" dans la région "Montérégie"

# # Convertir la colonne 'add_date' au format datetime
# mont_regie_maisons["add_date"] = pd.to_datetime(mont_regie_maisons["add_date"])

# # Ajouter des colonnes pour l'année et le mois
# mont_regie_maisons["year"] = mont_regie_maisons["add_date"].dt.year
# mont_regie_maisons["month"] = mont_regie_maisons["add_date"].dt.month

# # Calculer les tendances annuelles et mensuelles
# monthly_trends = mont_regie_maisons.groupby(["year", "month"]).agg(
#     prix_moyen=("price", "mean"),
#     prix_median=("price", "median"),
#     nb_maisons=("price", "count"),
#     prix_max=("price", "max"),
#     prix_min=("price", "min"),
# ).reset_index()

# # Visualisation 1: Évolution mensuelle et annuelle des prix moyens
# plt.figure(figsize=(12, 6))
# sns.lineplot(data=monthly_trends, x="month", y="prix_moyen", hue="year", marker="o")
# plt.title("Évolution mensuelle des prix moyens par année")
# plt.xlabel("Mois")
# plt.ylabel("Prix moyen ($)")
# plt.legend(title="Année")
# plt.grid(alpha=0.3)
# plt.show()

# # Visualisation 2: Distribution des prix par mois (boxplot)
# plt.figure(figsize=(12, 6))
# sns.boxplot(data=mont_regie_maisons, x="month", y="price", palette="viridis")
# plt.title("Distribution des prix par mois")
# plt.xlabel("Mois")
# plt.ylabel("Prix ($)")
# plt.grid(alpha=0.3)
# plt.show()

# # Visualisation 3: Histogramme des prix pour détecter les anomalies
# plt.figure(figsize=(12, 6))
# sns.histplot(mont_regie_maisons["price"], bins=50, kde=True, color="skyblue")
# plt.title("Distribution des prix pour les maisons en Montérégie")
# plt.xlabel("Prix ($)")
# plt.ylabel("Nombre de maisons")
# plt.grid(alpha=0.3)
# plt.show()

# # Visualisation 4: Analyse des prix en fonction des surfaces
# plt.figure(figsize=(12, 6))
# sns.scatterplot(
#     data=mont_regie_maisons,
#     x="surface",
#     y="price",
#     hue="nb_bedroom",
#     palette="coolwarm",
#     alpha=0.7,
# )
# plt.title("Prix en fonction de la surface et du nombre de chambres")
# plt.xlabel("Surface (m²)")
# plt.ylabel("Prix ($)")
# plt.legend(title="Nombre de chambres")
# plt.grid(alpha=0.3)
# plt.show()